# Homework 5: Lexicons and Distributional Semantics

This is due on **Friday, 11/10 (11pm)** 

## How to do this problem set

Most of these questions require writing Python code and computing results, and the rest of them have textual answers.  Write all the textual answers in this document, show the output of your experiment in this document, and implement the functions in the `python` files. 

Submit a PDF of thie .ipynb to Gradescope, and the .ipynb and all python files to Moodle.

The assignment has two parts:
 * In the first part, you will experiment with Turney's method to find word polarities in a twitter dataset, given some positive and negative seed words.
 * In the second part, you will experiment with distributional and vector semantics.

**Your Name: <>**

**List collaborators, and how you collaborated, here:** (see our [grading and policies page](http://people.cs.umass.edu/~brenocon/inlp2017/grading.html) for details on our collaboration policy).

* _name 1_ 

## Part 1: Lexicon semantics


Recall that PMI of a pair of words, is defined as:

$$PMI(x, y) = log\frac{ P(x, y) }{ P(x)P(y)}$$

The Turney mehod defines a word's polarity as:

$$Polarity(word) = PMI(word, positive\_word)−PMI(word, negative\_word)$$

where the joint probability $P(w, v)$ or, more specifically, $P(w\ NEAR\ v)$ is the probability of both being "near" each other.  We'll work with tweets, so it means: if you choose a tweet at random, what's the chance it contains both `w` and `v`?

(If you look at the Turney method as explained in the SLP3 chapter, the "hits" function is a count of web pages that contain at least one instance of the two words occurring near each other.)

The positive_word and negative_word terms are initially constructed by hand. For example: we might start with single positive word ('excellent') and a single negative word ('bad'). We can also have list of positive words ('excellent', 'perfect', 'love', ....) and list of negative words ('bad', 'hate', 'filthy',....)

If we're using a seed list of multiple terms, just combine them into a single symbol, e.g. all the positive seed words get rewritten to POS_WORD (and similarly for NEG_WORD).  This $P(w, POS\_WORD)$ effectively means the co-ocurrence of $w$ with any of the terms in the list.

For this assignment, we will use twitter dataset which has 349378 tweets. These tweets are in the file named `tweets.txt`. These are the tweets of one day and filtered such that each tweet contains at least one of the seed words we've selected.

## Question 1 (15 points)

The file `tweets.txt` contains around 349,378 tweets with one tweet per line.  It is a random sample of public tweets, which we tokenized with [twokenize.py's tokenizeRawTweetText()](https://github.com/myleott/ark-twokenize-py/blob/master/twokenize.py)). The text you see has a space between each token so you can just use `.split()` if you want.  We also filtered tweets to ones that included at least one term from one of these seed lists:
* Positive seed list: ["good", "nice", "love", "excellent", "fortunate", "correct", "superior"]
* Negative seed list: ["bad", "nasty", "poor", "hate", "unfortunate", "wrong", "inferior"]

Each tweet contains at least one positive or negative seed word. Take a look at the file (e.g. `less' and `grep'). Implement the Turney's method to calculate polarity scores of all words.

Some things to keep in mind:
* Ignore the seed words (i.e. don't calculate the polarity of the seed words).
* You may want to ignore the polarity of words beignning with `@` or `#`. 

We recommend that you write code in a python file, but it's up to you.

QUESTION: You'll have to do something to handle zeros in the PMI equation. Please explain your and justify your decision about this.

**textual answer here**

## Question 2 (5 points)

Print the top 50 most-positive words (i.e. inferred positive words) and the 50 most-negative words.

Many of the words won't make sense.  Comment on at least two that do make sense, and at least two that don't.  Why do you think these are happening with this dataset and method?

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
# Write code to print words here
import question1
from collections import defaultdict

POS_WORDS = ["good", "nice", "love", "excellent", "fortunate", "correct", "superior"]
NEG_WORDS = ["bad", "nasty", "poor", "hate", "unfortunate", "wrong", "inferior"]
result = question1.readTweets()    


In [93]:
word_counts = defaultdict(int)
for i,tweet in enumerate(result):
    for word in set(tweet):
        word_counts[word] += 1
    
cooccurence_count = {}
for pos_word in POS_WORDS:
    cooccurence_count[pos_word] = defaultdict(int)
for neg_word in NEG_WORDS:
    cooccurence_count[neg_word] = defaultdict(int)

for i,tweet in enumerate(result):
    for word in set(tweet):
        if (word not in POS_WORDS) or (word not in NEG_WORDS):
            for pos_word in POS_WORDS:
                if pos_word in tweet:
                    cooccurence_count[pos_word][word]+=1
            for neg_word in NEG_WORDS:
                if neg_word in tweet:
                    cooccurence_count[neg_word][word]+=1

In [90]:
import math
polarity = {}
for word in word_counts:
    if not (word in POS_WORDS or word in NEG_WORDS or word[0] in ['@','#']):
        pmi_pos = 0   
        for pos_word in POS_WORDS: 
            numerator = (1 + cooccurence_count[pos_word][word])*1.0/len(result)
            denominator = ((1.0 + word_counts[word])/len(result))\
            *((1.0 + word_counts[pos_word])/len(result))
            pmi_pos += math.log(numerator/denominator)
        
        pmi_neg = 0   
        for neg_word in NEG_WORDS: 
            numerator = (1 + cooccurence_count[neg_word][word])*1.0/len(result)
            denominator = ((1.0 + word_counts[word])/len(result))\
                        *((1.0 + word_counts[neg_word])/len(result))
            pmi_neg += math.log(numerator/denominator)
        
        polarity[word] = pmi_pos - pmi_neg

In [91]:
import operator
most_positive = sorted(polarity.items(), key=operator.itemgetter(1), reverse=True)
most_negative = sorted(polarity.items(), key=operator.itemgetter(1))
print "Most Positive Words\n",most_positive[0:50]
print "\nMost Negative Words", most_negative[0:50]


Most Positive Words
[('evening', 11.861793364063999), ('craze', 9.86355135912765), ('birthday', 9.63620410765375), ('hello', 8.955751095668278), ('thanks', 8.474708091435518), ('thank', 8.444648424158029), ('usatour2017', 8.258610367443737), ('alwaysjadine', 8.25139011947025), ('lauren', 7.972278755492287), ('nnpanalomoto', 7.8580923484190315), ('23', 7.824778543518079), ('wonderful', 7.458687417013186), ('lovely', 7.355387847175383), ('flowers', 7.351622597576826), ('goodbye', 7.256542012538748), ('tshirt', 7.221856454550858), ('addition', 7.2100219969038575), ('thx', 7.126945354767376), ('doggo', 7.120223809007047), ('congratulations', 7.116184654843826), ('ol', 7.104073418894475), ('dropping', 7.098680753314957), ('movie', 7.086012359294896), ('justin', 7.059920840880875), ('london', 6.979260214704757), ('congrats', 6.966964204922071), ('march', 6.921635946950281), ('von', 6.912083424284884), ('viola', 6.857146111812365), ("they'd", 6.828087386205887), ('nd', 6.823759095779629), ('x

### Textual answer here.
In positive words, I can see worderful, happy, thanks, goodbye, hello which make sense. Words like nd, tak don't make sense. It is happening mostly beacause the word splitting is bad.

Same with the most negative words. Words like crimes, victims, violence, blam make sense. but word like 3am does not signify anything.

## Question 3 (5 points)

Now filter out all the words which have total count < 500, and then print top 50 polarity words and bottom 50 polarity words. 

Choose some of the words from both the sets of 50 words you got above which accoording to you make sense. Again please note, you will find many words which don't make sense. Do you think these results are better than the results you got in Question-1? Explain why.

In [92]:
# Write code to print words here
count = 0
print "\n******* Most Positive Words after filetering *******\n"
for word,val in most_positive:
    if word_counts[word] > 500:
        print word, val
        count+=1
    if count > 50:
        break

count = 0
print "\n******* Most Negative Words after filetering *******\n"
for word,val in most_negative:
    if word_counts[word] > 500:
        print word, val
        count+=1
    if count > 50:
        break


        
        


******* Most Positive Words after filetering *******

birthday 9.63620410765
thanks 8.47470809144
thank 8.44464842416
ol 7.10407341889
movie 7.08601235929
march 6.92163594695
von 6.91208342428
hood 6.50890664669
morning 6.42584459989
taylor 6.23145775052
happy 6.1301655718
teyana 5.97982006379
keef 5.81347113562
herban 5.81232500369
httpstcoi46mhn5tag 5.81232500369
httpstcoinu4pqos7z 5.79846796902
retweet 5.67531973081
hi 5.67139977819
president 5.64659953579
proud 5.55009241126
sweet 5.46412309759
best 5.42007300529
soon 5.38470895571
amazing 5.29710361876
follow 5.17393859191
far 5.16435740514
together 5.03591425461
beautiful 4.98297931669
miss 4.88006546877
pregnancy 4.8563292416
heard 4.84223394064
today 4.82133269829
win 4.76449437313
music 4.7615063252
learn 4.72999439257
lt 4.71102441138
work 4.66648052359
speech 4.5941589376
luck 4.57704500464
friends 4.52279223327
fall 4.50458175792
positive 4.43200885152
giving 4.3345503865
yes 4.3061930236
sounds 4.29789268702
d 4.296962698

### Textual answer here.

## Question 4 (5 points)

Even after filtering out words with count < 500, many top-most and bottom-most polarity don't make sense. Identify what kind of words these are and what can be done to filter them out. You can read some tweets in the file to see what's happening. 

### Textual answer here.
In positive words, there are many re-tweets about "That 's good ol Keef Von D , or Herban Decay". Here the word ol comes algong with good many times and hence its polarity is very high.

Same is applicable for word "saf" in most negative words. There are many re-tweets about "CHILD : Why is it bad to have marijuana in every store ? ME : I’ll explain after I pick up cigarettes and beer at Saf…" and hence saf gets high cooccurence probability with word bad.


We need to filter out re-tweets and just keep the database about all the unique tweets.

# Part-2: Distributional Semantics

## Cosine Similarity

Recall that, where $i$ indexes over the context types, cosine similarity is defined as follows. $x$ and $y$ are both vectors of context counts (each for a different word), where $x_i$ is the count of context $i$.

$$cossim(x,y) = \frac{ \sum_i x_i y_i }{ \sqrt{\sum_i x_i^2} \sqrt{\sum_i y_i^2} }$$

The nice thing about cosine similarity is that it is normalized: no matter what the input vectors are, the output is between 0 and 1. One way to think of this is that cosine similarity is just, um, the cosine function, which has this property (for non-negative $x$ and $y$). Another way to think of it is, to work through the situations of maximum and minimum similarity between two context vectors, starting from the definition above.

Note: a good way to understand the cosine similarity function is that the numerator cares about whether the $x$ and $y$ vectors are correlated. If $x$ and $y$ tend to have high values for the same contexts, the numerator tends to be big. The denominator can be thought of as a normalization factor: if all the values of $x$ are really large, for example, dividing by the square root of their sum-of-squares prevents the whole thing from getting arbitrarily large. In fact, dividing by both these things (aka their norms) means the whole thing can’t go higher than 1.

In this problem we'll work with vectors of raw context counts.  (As you know, this is not necessarily the best representation.)

## Question 5 (5 points)

See the file `nytcounts.university_cat_dog`, which contains context count vectors for three words: “dog”, “cat”, and “university”. These are immediate left and right contexts from a New York Times corpus. You can open the file in a text editor since it’s quite small. 

Write a function which takes context count dictionaries of two words and calculates cosine similarity between these two words. The function should return a number beween 0 and 1.  Briefly comment on whether the relative simlarities make sense.



In [102]:
import distsim; 
reload(distsim)

word_to_ccdict = distsim.load_contexts("nytcounts.university_cat_dog")

# write code here to show output (i.e. cosine similarity between these words.)
# We encourage you to write other functions in distsim.py itself. 
print distsim.cosine_similarity(word_to_ccdict['dog'],word_to_ccdict['cat'])
print distsim.cosine_similarity(word_to_ccdict['cat'],word_to_ccdict['university'])
print distsim.cosine_similarity(word_to_ccdict['dog'],word_to_ccdict['university'])

file nytcounts.university_cat_dog has contexts for 3 words
0.966891672715
0.660442421144
0.659230248969


**Write your response here:**





## Question 6 (20 points)

Explore similarities in `nytcounts.4k`, which contains context counts for about 4000 words in a sample of New York Times articles. The news data was lowercased and URLs were removed. The context counts are for the 2000 most common words in twitter, as well as the most common 2000 words in the New York Times. (But all context counts are from New York Times.) The context counts only contain contexts that appeared for more than one word.  The file has three tab-separate fields: the word, its count, and a JSON-encoded dictionary of its context counts.  You'll see it's just counts of the immediate left/right neighbors.

Choose **six** words. For each, show the output of 20 nearest words (use cosine similarity as distance metric). Comment on whether the output makes sense. Comment on whether this approach to distributional similarity makes more or less sense for certain terms.
Four of your words should be:

 * a name (for example: person, organization, or location)
 * a common noun
 * an adjective
 * a verb

You may also want to try exploring further words that are returned from a most-similar list from one of these. You can think of this as traversing the similarity graph among words.

*Implementation note:* 
On my laptop it takes several hundred MB of memory to load it into memory from the `load_contexts()` function. If you don’t have enough memory available, your computer will get very slow because the OS will start swapping. If you have to use a machine without that much memory available, you can instead implement in a streaming approach by using the `stream_contexts()` generator function to access the data; this lets you iterate through the data from disk, one vector at a time, without putting everything into memory. You can see its use in the loading function. (You could also alternatively use a key-value or other type of database, but that’s too much work for this assignment.)

In [103]:
import distsim; reload(distsim)
word_to_ccdict = distsim.load_contexts("nytcounts.4k")
###Provide your answer below; perhaps in another cell so you don't have to reload the data each time

file nytcounts.4k has contexts for 3648 words


In [107]:
test_words = ['bill','awake','established','christ','told','boobs']
result_dict = {}
for word in test_words:
    result_dict[word] = {}
    for compare_word in word_to_ccdict.keys():
        if compare_word != word:
            result_dict[word][compare_word] = distsim.cosine_similarity(word_to_ccdict[word]\
                                            ,word_to_ccdict[compare_word])

In [111]:
for word in result_dict.keys():
    print "\n******** Most similar words for "+word.upper()+" ********\n"
    sorted_x = sorted(result_dict[word].items(), key=operator.itemgetter(1), reverse=True)[0:20]
    for t in sorted_x:
        print t[0],t[1]

    


******** Most similar words for ESTABLISHED ********

introduced 0.866978464017
created 0.826531601085
built 0.824161262039
developed 0.819753428508
played 0.805059352133
raised 0.80203725391
cited 0.800262031243
filed 0.787898026519
founded 0.785600043102
completed 0.784712932447
represented 0.772626786416
performed 0.770721128552
provided 0.764642928569
published 0.764630929142
approved 0.755176119183
opened 0.753481348476
brought 0.748781057709
rejected 0.747913554615
conducted 0.742548463328
playing 0.737244996042

******** Most similar words for CHRIST ********

course 0.778852656907
flowers 0.755570700199
pennsylvania 0.755532934775
mine 0.740786349894
directors 0.739045888437
massachusetts 0.725433110728
parliament 0.720827327631
california 0.719758096072
art 0.71908435029
education 0.718813872706
medicine 0.718375772736
color 0.711648443544
virginia 0.703757104294
technology 0.700545583676
violence 0.697304765008
wood 0.697304764974
oz 0.696178786893
texas 0.695126105149
archi

## Question 7 (10 points)

In the next several questions, you'll examine similarities in trained word embeddings, instead of raw context counts.

See the file `nyt_word2vec.university_cat_dog`, which contains word embedding vectors pretrained by word2vec [1] for three words: “dog”, “cat”, and “university”, from the same corpus. You can open the file in a text editor since it’s quite small.

Write a function which takes word embedding vectors of two words and calculates cossine similarity between these 2 words. The function should return a number beween -1 and 1. Briefly comment on whether the relative simlarities make sense. 

*Implementation note:*
Notice that the inputs of this function are numpy arrays (v1 and v2). If you are not very familiar with the basic operation in numpy, you can find some examples in the basic operation section here:
https://docs.scipy.org/doc/numpy-dev/user/quickstart.html

If you know how to use Matlab but haven't tried numpy before, the following link should be helpful:
https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html

[1] Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." NIPS 2013.

In [9]:
import distsim; reload(distsim)

word_to_vec_dict = distsim.load_word2vec("nyt_word2vec.4k")

print distsim.cosine_similarity_word2vec(word_to_vec_dict['dog'],word_to_vec_dict['cat'])
print distsim.cosine_similarity_word2vec(word_to_vec_dict['cat'],word_to_vec_dict['university'])
print distsim.cosine_similarity_word2vec(word_to_vec_dict['dog'],word_to_vec_dict['university'])

# write code here to show output (i.e. cosine similarity between these words.)
# We encourage you to write other functions in distsim.py itself. 

0.827517295965
-0.205394745036
-0.190753135501


**Write your response here:**




## Question 8 (20 points)

Repeat the process you did in the question 6, but now use dense vector from word2vec. Comment on whether the outputs makes sense. Compare the outputs of using nearest words on word2vec and the outputs on sparse context vector (so we suggest you to use the same words in question 6). Which method works better on the query words you choose. Please brief explain why one method works better than other in each case.

Not: we used the default parameters of word2vec in [gensim](https://radimrehurek.com/gensim/models/word2vec.html) to get word2vec word embeddings.

In [15]:
import distsim
import operator
word_to_vec_dict = distsim.load_word2vec("nyt_word2vec.4k")

###Provide your answer below
test_words = ['bill','awake','established','christ','told','kentucky']
result_dict = {}
for word in test_words:
    result_dict[word] = {}
    for compare_word in word_to_vec_dict.keys():
        if compare_word != word:
            result_dict[word][compare_word] = distsim.cosine_similarity_word2vec(word_to_vec_dict[word]\
                                            ,word_to_vec_dict[compare_word])

In [16]:
for word in result_dict.keys():
    print "\n******** Most similar words for "+word.upper()+" ********\n"
    sorted_x = sorted(result_dict[word].items(), key=operator.itemgetter(1), reverse=True)[0:20]
    for t in sorted_x:
        print t[0],t[1]


******** Most similar words for ESTABLISHED ********

created 0.768407638864
developed 0.741681884818
founded 0.719516204749
joined 0.670916418553
represented 0.661429266785
organized 0.649323577778
built 0.61853399759
introduced 0.614364798068
hired 0.593882369557
conducted 0.578460391706
supported 0.575789765017
designed 0.569701186273
presented 0.567261554753
chosen 0.557226914843
worked 0.552443411205
produced 0.552137832057
performed 0.545909327103
replaced 0.545069865237
criticized 0.534712729592
accepted 0.527069712429

******** Most similar words for CHRIST ********

jesus 0.724308667655
church 0.607908689327
christian 0.579388667164
holy 0.575210234377
mary 0.565814213229
jewish 0.564763204248
roman 0.548141728208
queen 0.538960423705
elizabeth 0.499477936788
ceremony 0.485140946065
catholic 0.481478696692
god 0.479151790923
lord 0.472479178139
rev 0.465089432227
lady 0.461842290819
un 0.457497369631
mt 0.443250262586
soul 0.438405232957
king 0.437203017882
faith 0.4370979993

## Question 9 (15 points)
An interesting thing to try with word embeddings is analogical reasoning tasks. In the following example, it's intended to solve the analogy question "king is to man as what is to woman?", or in SAT-style notation,

king : man :: ____ : woman

Some research has proposed to use additive operations on word embeddings to solve the analogy: take the vector $(v_{king}-v_{man}+v_{woman})$ and find the most-similar word to it.  One way to explain this idea: if you take "king", get rid of its attributes/contexts it shares with "man", and add in the attributes/contexts of "woman", hopefully you'll get to a point in the space that has king-like attributes but the "man" ones replaced with "woman" ones.

Show the output for 20 closest words you get by trying to solve that analogy with this method.  Did it work?

Please come up with another analogical reasoning task (another triple of words), and output the answer using the same method. Comment on whether the output makes sense. If the output makes sense, explain why we can capture such relation between words using an unsupervised algorithm. Where does the information come from? On the other hand, if the output does not make sense, propose an explanation why the algorithm fails on this case.


Note that the word2vec is trained in an unsupervised manner just with distributional statistics; it is interesting that it can apparently do any reasoning at all.  For a critical view, see [Linzen 2016](http://www.aclweb.org/anthology/W/W16/W16-2503.pdf).



In [21]:
# Write code to show output here.
from operator import add
from operator import sub
target = map(add, map(sub, word_to_vec_dict['king'], word_to_vec_dict['man']),\
             word_to_vec_dict['woman'])

shortest = 10000
best_word = ""
for word in word_to_vec_dict.keys():
    dist = distsim.cosine_similarity_word2vec(word_to_vec_dict[word], target) < shortest
    if dist < shortest:
        shortest = dist
        best_word = word

print best_word

limited


### Textual answer here.